In [ ]:
#THIS FUNCTION IS FOR RUNNING WITH SLURM JOB ARRAY
#(SPLITS UP JOB_ARRAY BELOW INTO EVEN MORE TASKS)
def StartSlurmJobArray(num_jobs,num_slurm_jobs, ISRUN):
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=1
    if ISRUN==False:
        start_job=1;end_job=num_jobs+1
        return start_job,end_job
    total_elements=num_jobs #total num of variables

    job_range = total_elements // num_slurm_jobs  # Base size for each chunk
    remaining = total_elements % num_slurm_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_slurm_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_slurm_jobs - 1: 
            end_job = total_elements 
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_slurm_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    # if sbatch==True:
        
    start_job, end_job = get_job_range(job_id, num_slurm_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    if start_job==0: start_job=1
    if end_job==total_elements: end_job+=1
    return start_job,end_job

# job_id=1
# [start_slurm_job,end_slurm_job,slurm_index_adjust]=StartSlurmJobArray(num_jobs,num_slurm_jobs,ISRUN)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# dx = 1 km; Np = 1M; Nt = 5 min
data1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
parcel1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
res='1km';t_res='5min'
Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 100M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_100M.nc') #***
# res='1km'; t_res='1min'; Np_str='100e6'

# #uncomment if using 250m data
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_250m.nc') #***
# # # parcel1=xr.open_dataset(dir2+'cm1out_pdata_250m.nc') #***

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
####################################
# SETUP

In [ ]:
################################
#ENTRAINMENT DATA_TYPE SETUP
# PROCESSING=False #if using full entrainment calculations
PROCESSING=True #if using turbulence-filtered entrainment calculations
################################

In [ ]:
################################
#JOB ARRAY SETUP
################################
#*#*
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
num_jobs=60 #1M parcels
# num_jobs=200 #50M parcels

num_slurm_jobs=60
##############################

In [ ]:
##############################################
#DATA LOADING FUNCTIONS

In [ ]:
#INITIALIZE DATA FUNCTION
###############################################################
def InitiateArray(out_file,vars,t_chunk_size,z_chunk_size,t_size=None,z_size=None):
    # Define array dimensions (adjust based on your data)

    if t_size==None:
        t_size = len(data1['time'])  # Number of timesteps
    if z_size==None:
        z_size = len(data1['zh'])    # Number of vertical levels
    
    with h5py.File(out_file, 'w') as f: 
        # Check if the dataset 'theta_e' already exists
        for var_name in vars:
            if var_name not in f:
                # Create a dataset with the full size for all time steps (initially empty)
                f.create_dataset(var_name, 
                                 (t_size, z_size),  # Full size for all timesteps
                                 chunks=(t_chunk_size, z_chunk_size))  # Chunks for time axis to allow resizing

def InitiateArray_Job(data,out_file,vars,t_chunk_size,z_chunk_size,t_size=None,z_size=None):
    # Define array dimensions (adjust based on your data)

    if t_size==None:
        t_size = len(data['time'])  # Number of timesteps
    if z_size==None:
        z_size = len(data1['zh'])    # Number of vertical levels
    
    with h5py.File(out_file, 'w') as f: 
        # Check if the dataset 'theta_e' already exists
        for var_name in vars:
            if var_name not in f:
                # Create a dataset with the full size for all time steps (initially empty)
                f.create_dataset(var_name, 
                                 (t_size, z_size),  # Full size for all timesteps
                                 chunks=(t_chunk_size, z_chunk_size))  # Chunks for time axis to allow resizing

In [ ]:
#JOB ARRAY SETUP
def StartJobArray(job_id,num_jobs):
    total_elements=len(data1['time']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
    #     job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    #     if job_id==0: job_id=1
        
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    return start_job,end_job,index_adjust
# job_id=1
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)

In [ ]:
# Reading Back Data Later
##############
def make_data_dict(in_file,var_names,read_type,start_job,end_job):
    if read_type=='h5py':
        with h5py.File(in_file, 'r') as f:
            data_dict = {var_name: f[var_name][start_job:end_job] for var_name in var_names}

            
    elif read_type=='xarray':
        in_data = xr.open_dataset(
            in_file,
            engine='h5netcdf',
            phony_dims='sort',
            chunks={'phony_dim_0': 100, 'phony_dim_1': 1_000_000} 
        )
        data_dict = {k: in_data[k][start_job:end_job].compute().data for k in var_names}

    return data_dict

# read_type='xarray'
read_type='h5py'

In [ ]:
def GetData(data1,parcel1,start_job,end_job):
    #Indexing Array with JobArray
    data=data1.isel(time=slice(start_job,end_job))
    parcel=parcel1.isel(time=slice(start_job,end_job))
    return data,parcel
# [data,parcel]=GetData(data1,parcel1,start_job,end_job)

In [ ]:
def GetSpatialData(data,start_job,end_job):
    import h5py
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['Z']#, 'Y', 'X']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    Z, = (data_dict[k] for k in var_names) #,Y,X
    
    # #Making Time Matrix
    Nt=len(data['time'])
    T = np.broadcast_to(np.arange(Nt)[:, None], A_c.shape)  # shape: (Nt, p)
    
    # check_memory(globals())
    return T,Z#,Y,X

In [ ]:
def GetAData(PROCESSING,start_job,end_job):
    if PROCESSING==False:
        dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
        in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
        
        var_names = ['A_g', 'A_c']
        data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
        A_g, A_c = (data_dict[k] for k in var_names) #, W
    
    elif PROCESSING==True:
        dir2=dir+'Project_Algorithms/Entrainment/'
        in_file=dir2+f'processed_binary_arrays_{res}_{t_res}_{Np_str}.h5'
        
        var_names = ['A_g_Processed', 'A_c_Processed']
        data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
        A_g,A_c = (data_dict[k] for k in var_names)
        # check_memory(globals())
    return A_g,A_c

In [ ]:
def GetAPrior(PROCESSING,start_job): #FIX FOR JOB_ARRAY (SO FIRST TIMESTEP WILL NOT BE BLANK)
    if start_job!=0:
        A_g_Prior,A_c_Prior=GetAData(PROCESSING,start_job-1,start_job);
        return A_g_Prior,A_c_Prior
    else:
        return 0,0

In [ ]:
###########################################################################################################################################################################

In [ ]:
####################################
#Functions

In [ ]:
def SubtractA(A,A_Prior):
    D = np.zeros_like(A,dtype=np.int8)
    D[1:, :] = A[1:, :]*1 - A[:-1, :]*1
    if start_job!=0:
        D[0, :] = A[0, :]*1-A_Prior*1 #FIX FOR FIRST TIMESTEP
    return D

def AddAs(A1,A2,A2_Prior):
    S = np.zeros_like(A1,dtype=np.int8)
    S[1:, :] = A1[1:, :]*1 + A2[:-1, :]*1
    if start_job!=0:
        S[0, :] = A1[0, :]*1 + A2_Prior*1 #FIX FOR FIRST TIMESTEP
    return S

def Heavyside(D,type):
    if type=='e':
        D[D < 0] = 0
    elif type=='d':
        D[D > 0] = 0
        D*=-1 #not in main entrainment code (where it is not necessary), here only 0s and 1s can be used
    return D

def ed2d(data, A1,A2, T, Z, type,type2, A1_Prior,A2_Prior): #*#*
    # start_time = time.time()
    """
    Function to compute 2D entrainment and update result array based on provided inputs.
    
    Returns a 2D (t,z) array containing the sum of the D array representing entrained parcels, by 1, and detrained parcels, by -1.
    The finally array is then ordered by the appropiate index using the np.add.at function
    
    Parameters:
    - A: The (t,p) lagrangian binary array.
    - T: The (t,p) lagrangian time index array.
    - Z: The (t,p) Lagrangian z index array.
    """
    # Compute the difference between neighboring elements along the first axis
    D1=SubtractA(A1,A1_Prior); D2=SubtractA(A2,A2_Prior)

    # Update D for entrainment/detrainment with the Heavyside function
    D1=Heavyside(D1,type); D2=Heavyside(D2,type)

    # Getting Combined Entrainment/Detrainment Area
    if type2=='general':
        D=D1
    elif type2=='cloudy':
        D=D2
    
    #removing entrainment going from general <==> cloudy updrafts
    #must be either entrained OR detrained from/to environment) AND (NOT move between general and cloudy updrafts)
    S1=AddAs(A1,A2,A2_Prior); S2=AddAs(A2,A1,A1_Prior)
    cond1=(S1!=2) #General <==> Cloudy
    cond2=(S2!=2) #Cloudy <==> General
    D = D & (cond1&cond2)

    # Initialize time and vertical dimension arrays
    Nt = len(data['time']); Nz = len(data1['zh'])
    # Initialize result array
    result = np.zeros((Nt, Nz))

    # Use np.add.at to accumulate values in the result array
    np.add.at(result, (T, Z), D)

    # end_time = time.time()
    # print(f"Execution time: {(end_time - start_time)} seconds")
    return result

In [ ]:
def MakeDictionary(**vars):
    return vars
def RunCalculation(data, A_g,A_c, T,Z, A_g_Prior, A_c_Prior):    
    # Set A based on PROCESSING state
    
    profile_array_e_g = ed2d(data, A_g,A_c, T, Z, type='e',type2='general', A1_Prior=A_g_Prior, A2_Prior=A_c_Prior)
    profile_array_d_g = ed2d(data, A_g,A_c, T, Z, type='d',type2='general', A1_Prior=A_g_Prior, A2_Prior=A_c_Prior)
    profile_array_e_c = ed2d(data, A_g,A_c, T, Z, type='e',type2='cloudy', A1_Prior=A_g_Prior, A2_Prior=A_c_Prior)
    profile_array_d_c = ed2d(data, A_g,A_c, T, Z, type='d',type2='cloudy', A1_Prior=A_g_Prior, A2_Prior=A_c_Prior)

    VARs=MakeDictionary(profile_array_e_g=profile_array_e_g, profile_array_d_g=profile_array_d_g,
                        profile_array_e_c=profile_array_e_c, profile_array_d_c=profile_array_d_c)
    return VARs

In [ ]:
def SaveProfile(VARs,job_id,PROCESSING):
    dir2=dir+'Project_Algorithms/Entrainment/job_out_3/'
    
    #SAVING
    # print('SAVING')
    if PROCESSING==False:
        out_file=dir2+f'2D_entrainmentdetrainment_combined2_profiles_{res}_{t_res}_{Np_str}'
    elif PROCESSING==True:
        out_file=dir2+f'2D_entrainmentdetrainment_combined2_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}'
    out_file+=f'_{job_id}.h5'
    
    vars=VARs.keys()
    t_chunk_size=end_job-start_job
    z_chunk_size=2
    InitiateArray_Job(data,out_file,vars,t_chunk_size=t_chunk_size,z_chunk_size=z_chunk_size)
    
    with h5py.File(out_file, 'a') as f: 
        for var in VARs:
            f[f'{var}'][:]=VARs[var]
    # print('done')

In [ ]:
##############################################
#RUNNING

In [ ]:
[start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=False) #if ISRUN is False, then will not run using slurm_job_array

print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job-1)}")

job_id_list=np.arange(start_slurm_job,end_slurm_job)
for job_id in job_id_list:
    if job_id % 1 == 0: print(f'current job_id = {job_id}')
    [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)

    #SLICING DATA
    [data,parcel]=GetData(data1,parcel1,start_job,end_job)
    [A_g,A_c]=GetAData(PROCESSING,start_job,end_job)
    [A_g_Prior,A_c_Prior]=GetAPrior(PROCESSING,start_job) #FIX FOR FIRST TIMESTEP
    [T,Z]=GetSpatialData(data,start_job,end_job)

    #RUNNING CALCULATION
    VARs=RunCalculation(data, A_g,A_c, T,Z, A_g_Prior,A_c_Prior)

    #SAVING
    SaveProfile(VARs,job_id,PROCESSING)
    # check_memory(globals())
    # del VARs

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE IF JOB ARRAY IS RUNNING
recombine=True

In [ ]:
def Recombine(PROCESSING,num_jobs):
    dir2=dir+'Project_Algorithms/Entrainment/job_out_3/'
    dir3=dir+'Project_Algorithms/Entrainment/'
    
    if PROCESSING==False:
        out_file=dir3+f'2D_entrainmentdetrainment_combined2_profiles_{res}_{t_res}_{Np_str}.h5'
    elif PROCESSING==True:
        out_file=dir3+f'2D_entrainmentdetrainment_combined2_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'
    
    vars=["profile_array_e_g","profile_array_d_g",
          "profile_array_e_c","profile_array_d_c"]
    InitiateArray(out_file,vars,t_chunk_size=50,z_chunk_size=2)
    
    with h5py.File(out_file, 'r+') as f_out:
        for job_id in np.arange(1,num_jobs+1):
            if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
            [a,b,_]=StartJobArray(job_id,num_jobs)
    
            if PROCESSING==False:
                in_file=dir2+f'2D_entrainmentdetrainment_combined2_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5'
            elif PROCESSING==True:
                in_file=dir2+f'2D_entrainmentdetrainment_combined2_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}_{job_id}.h5'
            with h5py.File(in_file, 'r') as f_in: 
                for var in vars:
                    f_out[var][a:b]=f_in[var][:]

In [ ]:
if recombine==True:
    Recombine(PROCESSING,num_jobs=num_jobs)